In [ ]:


class Net(torch.nn.Module):
    def __init__(self, n_qubit):
        super(Net, self).__init__()
        self.n_qubit = n_qubit
        self.device = torch.device("cuda:0")
        self.params = torch.nn.Parameter(
            torch.tensor([[-0.1456], [0.1234], [2], [-0.34]]).to(self.device),
            requires_grad=True,
        )

    def forward(self):
        ansatz_gpu = Ansatz(self.n_qubit)
        ansatz_gpu.add_gate(X_tensor, 0)
        ansatz_gpu.add_gate(H_tensor, 0)
        ansatz_gpu.add_gate(Rxx_tensor(self.params[0]), [0, 1])
        ansatz_gpu.add_gate(Rxx_tensor(self.params[1]), [0, 2])
        # ansatz_gpu.add_gate(Rxx_tensor(self.params[2]), [0, 3])
        # ansatz_gpu.add_gate(Rxx_tensor(self.params[3]), [0, 4])
        ansatz_gpu.add_gate(H_tensor, 0)

        state, prob = gpu_forward(ansatz_gpu, self.n_qubit, readout=0)
        return state, prob


def seed(seed: int):
    """Set random seed.

    Args:
        seed (int): The random seed.
    """
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True


if __name__ == "__main__":
    from QuICT.algorithm.quantum_machine_learning.utils import Ansatz
    from QuICT.core import Circuit
    from QuICT.core.gate import *

    seed(0)
    device = torch.device("cuda:0")
    n_qubit = 3

    net = Net(n_qubit)
    optimizer = torch.optim.SGD
    optim = optimizer([dict(params=net.parameters(), lr=0.1)])
    net.train()

    for i in range(30):
        print("--------------------------------------")
        optim.zero_grad()
        state, prob = net()
        state.retain_grad()
        prob.retain_grad()

        bits_idx = [1 << i for i in range(n_qubit)]
        qid_idx = 1 << 0
        offset = list(set(bits_idx) - (set([qid_idx])))
        idx_0 = [0]
        idx_1 = [qid_idx]
        for i in range(len(offset)):
            for j in range(len(idx_1)):
                idx_0.append(offset[i] + idx_0[j])
                idx_1.append(offset[i] + idx_1[j])

        # Calculate probabilities
        prob_pytorch = torch.sum(torch.abs(state[idx_0]) * torch.abs(state[idx_0]))

        # loss = torch.abs(0.3 - prob_pytorch)
        loss = torch.abs(0.3 - prob)
        loss.backward()
        optim.step()
        print(net.params)
